In [4]:
## OpenWeatherMap API

# A nice person wrote a Python wrapper, which can be found here: https://github.com/csparpa/pyowm

import sys
!{sys.executable} -m pip install pyowm`
import request

ERROR: Invalid requirement: 'pyowm`'


ModuleNotFoundError: No module named 'request'

Create an account here: https://home.openweathermap.org/users/sign_up

Then Sign in and go to the "Keys" tab.

In [1]:
#create owm key 
OWM_key="7721c39fc555a1ad9edcdf327ad5385a"  

In [2]:
response=requests.get("http://api.openweathermap.org/data/2.5/air_pollution/history?lat=2.161165&lon=-41.689858&start=1622412000&end=1626040800&appid="+OWM_key )
response.status_code

NameError: name 'requests' is not defined

In [ ]:
response.json()

In [ ]:
import seaborn as sns

# nested dictionary!
data = response.json()

# get all data from it. First, explore the structure with .keys(), .items() etc.
# in order to know how to write the following lines

timestamps = [entry['dt'] for entry in response.json()['list']]
co_values = [entry['components']['co'] for entry in response.json()['list']]
no2_values = [entry['components']['no2'] for entry in response.json()['list']]
nh3_values = [entry['components']['nh3'] for entry in response.json()['list']]
o3_values = [entry['components']['o3'] for entry in response.json()['list']]


# build the flat dataframe
df = pd.DataFrame({'timestamps':timestamps,
                   'co_values':co_values,
                   'no2_values':no2_values,
                   'nh3_values':nh3_values,
                   'o3_values':o3_values,
                    # etc for more values 
                  })

# convert the UNIX time column to actual datetime
df['timestamps'] = pd.to_datetime(df['timestamps'], unit='s')

# conventional way of getting the data from that nested dictionary without list comprehension
# for entry in response.json()['list']:
#    print(entry['dt'])

### Plot

In [ ]:
# libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# Change the style of plot
plt.style.use('seaborn-darkgrid')
 
# Create a color palette
palette = plt.get_cmap('Set1')

fig, ax = plt.subplots(figsize=(14,10))
 
# Plot multiple lines
num=0
for column in df.drop('timestamps', axis=1):
    num+=1
    plt.plot(df['timestamps'],
             df[column],
             marker='',
             color=palette(num),
             linewidth=1,
             alpha=0.9,
             label=column,
            )

# Add legend
plt.legend(loc=2, ncol=2)
 
# Add titles
plt.title("A (bad) Spaghetti plot", loc='left', fontsize=12, fontweight=0, color='orange')
plt.xlabel("Time")
plt.ylabel("microgramm / m^3")

# Show the graph
plt.show()

In [ ]:
# less complicated using the pandas built in plot function - not the matplotlib one

df.set_index('timestamps')
df.plot.line(x='timestamps')

This is not a nice plot though, since the CO-values scatter over a way bigger range than all the other values, so it is better to create plots for each pollution variant or outsource at least CO to another diagram

In [ ]:
df.plot.line(x='timestamps',
             y=['no2_values', 'nh3_values'],
             ylabel="µg / m³",
             xlabel="time",
            figsize=(16,8));

In [ ]:
# plotting O3 and CO on another chart, because their order of magnitude is different
df.plot.line(x='timestamps',
             y=['o3_values', 'co_values'],
             color={"o3_values": "pink",
                    "co_values": "red"},
             ylabel="µg / m³",
             xlabel="time",
             figsize=(16,8)
            );